In [1]:
import pandas as pd
import requests
from datetime import datetime, timezone, timedelta
from io import StringIO
import geohash2

# Extraction

In [2]:
# Period time when sensors were validated
parameters = catalog.load('parameters')
start_datetime = parameters['start_datetime']
end_datetime = parameters['end_datetime']
print('start_datetime:', start_datetime)
print('end_datetime:', end_datetime)

2022-09-05 13:05:30,420 - kedro.io.data_catalog - INFO - Loading data from `parameters` (MemoryDataSet)...
start_datetime: 2022-09-04T00:00:00
end_datetime: 2022-09-04T23:59:59


In [3]:
# Request to InfluxDB API REST
def request_to_influxdb(sql_query):
    endpoint = "http://influxdb.canair.io:8086/query"
    database = "canairio"
    parameters = {
        'db': database,
        'q': sql_query,
        'epoch': 'ms'
    }
    # To get response as CSV text
    headers = {'Accept': 'application/csv'}
    # GET Request
    return requests.get(endpoint, params=parameters, headers=headers)

In [4]:
# Get Period Time
def get_period_time(start_datetime, end_datetime):
    start_datetime = int(datetime.fromisoformat(start_datetime).timestamp() * 1000)
    end_datetime = int(datetime.fromisoformat(end_datetime).timestamp() * 1000)
    return f"time >= {start_datetime}ms and time <= {end_datetime}ms"

In [5]:
# Period Time
period_time = get_period_time(start_datetime, end_datetime)
period_time

'time >= 1662267600000ms and time <= 1662353999000ms'

In [6]:
# Get SQL Query Tangaras
def get_sql_query_tangaras(start_datetime, end_datetime):
    sql_query = ""
    #period_time = "time >= now() - 1h and time <= now()"
    #period_time = "time >= now() - 24h and time <= now()"
    period_time = get_period_time(start_datetime, end_datetime)
    sql_query = "SELECT DISTINCT(geo) AS \"geohash\" "\
                "FROM \"fixed_stations_01\" WHERE "\
                    "(\"geo3\" = 'd29') AND "\
                    f"{period_time} "\
                "GROUP BY \"name\";"
    return sql_query

In [15]:
sql_query = get_sql_query_tangaras(start_datetime, end_datetime)
print(sql_query)

SELECT DISTINCT(geo) AS "geohash" FROM "fixed_stations_01" WHERE ("geo3" = 'd29') AND time >= 1662267600000ms and time <= 1662353999000ms GROUP BY "name";


In [7]:
# Get Data Frame Tangaras
def get_df_tangaras(start_datetime, end_datetime):
    # SQL Query Tangaras
    sql_query = get_sql_query_tangaras(start_datetime, end_datetime)
    # InfluxDB API REST Request
    influxdb_api_request = request_to_influxdb(sql_query)
    df_tangaras = pd.read_csv(StringIO(influxdb_api_request.text), sep=",")
    
    # Remove/Add Columns
    df_tangaras = df_tangaras[['tags', 'geohash']]
    df_tangaras['MAC'] = df_tangaras['tags'].apply(lambda x: x.split('=')[1])
    df_tangaras['GEOLOCATION'] = df_tangaras['geohash'].apply(lambda x: " ".join(str(value) for value in list(geohash2.decode_exactly(x)[0:2])))
    df_tangaras['LATITUDE'] = df_tangaras['GEOLOCATION'].apply(lambda x: x.split(' ')[0])
    df_tangaras['LONGITUDE'] = df_tangaras['GEOLOCATION'].apply(lambda x: x.split(' ')[1])
    df_tangaras['tags'] = df_tangaras['tags'].apply(lambda x: f"TANGARA_{x[-4:]}")
    df_tangaras.rename(columns={'tags': 'ID', 'geohash': 'GEOHASH'}, inplace=True)
    # Date time when query is executed
    tz = timezone(timedelta(hours=-5))
    df_tangaras['DATETIME'] = datetime.now(tz=tz)

    return df_tangaras

In [8]:
# Data Frame Tangaras
df_tangaras = get_df_tangaras(start_datetime, end_datetime)
df_tangaras.head()

,ID,GEOHASH,MAC,GEOLOCATION,LATITUDE,LONGITUDE,DATETIME
0,TANGARA_2BBA,d29e6b4,D29ESP32DE02BBA,3.3844757080078125 -76.51634216308594,3.3844757080078125,-76.51634216308594,2022-09-05 13:05:32.281197-05:00
1,TANGARA_14D6,d29dfx4,D29ESP32DED14D6,3.3350372314453125 -76.52732849121094,3.3350372314453125,-76.52732849121094,2022-09-05 13:05:32.281197-05:00
2,TANGARA_1CE2,d29e4cv,D29ESP32DED1CE2,3.3501434326171875 -76.51222229003906,3.3501434326171875,-76.51222229003906,2022-09-05 13:05:32.281197-05:00
3,TANGARA_1FCA,d29e48s,D29ESP32DED1FCA,3.3432769775390625 -76.52458190917969,3.3432769775390625,-76.52458190917969,2022-09-05 13:05:32.281197-05:00
4,TANGARA_2492,d29e64g,D29ESP32DED2492,3.3995819091796875 -76.54792785644531,3.3995819091796875,-76.54792785644531,2022-09-05 13:05:32.281197-05:00


In [9]:
# Save df_tangaras into Catalog
catalog.save('tangaras', df_tangaras)

2022-09-05 13:05:32,346 - kedro.io.data_catalog - INFO - Saving data to `tangaras` (CSVDataSet)...


In [10]:
# Get SQL Query Data Sensors
def get_sql_query_sensors(tangaras, start_datetime, end_datetime):
    sql_query = ""
    period_time = get_period_time(start_datetime, end_datetime)
    #period_time = "time >= now() - 1h and time <= now()"
    for mac in tangaras['MAC'].to_list():
        sql_query += "SELECT \"name\", last(\"pm25\") "\
                    "FROM \"fixed_stations_01\" WHERE "\
                    f"(\"name\" = '{mac}') AND "\
                    f"{period_time} " \
                    "GROUP BY time(30s) fill(none); "
    return sql_query[:-2]

In [11]:
# Get Data Frame Sensors
def get_df_sensors(tangaras, start_datetime, end_datetime):
    # Data Frame Sensors
    df_sensors = []
    # SQL Query Data Sensors
    sql_query = get_sql_query_sensors(tangaras, start_datetime, end_datetime)
    # InfluxDB API REST Request
    influxdb_api_request = request_to_influxdb(sql_query)
    df_influxdb_api_sensors = pd.read_csv(StringIO(influxdb_api_request.text), sep=",")

    # Remove/Add Columns
    df_influxdb_api_sensors = df_influxdb_api_sensors[['time', 'name.1', 'last']]
    df_influxdb_api_sensors.rename(columns={'time': 'DATETIME', 'name.1': 'MAC', 'last': 'PM25'}, inplace=True)

    # Truncate Response
    for index, row in tangaras.iterrows():
        df_sensor = df_influxdb_api_sensors.loc[df_influxdb_api_sensors['MAC'] == row['MAC']].reset_index(drop=True)[['DATETIME', 'PM25']] # Warning
        if not df_sensor.empty:
            df_sensor.rename(columns={'PM25': row['ID']}, inplace=True)
            df_sensor.set_index('DATETIME', inplace=True)
            df_sensors.append(df_sensor)
    
    df_sensors = df_sensors[0].join(df_sensors[1:]).reset_index()

    tz = timezone(timedelta(hours=-5))
    df_sensors['DATETIME'] = df_sensors['DATETIME'].apply(lambda x: datetime.fromtimestamp(int(x) / 1000, tz=tz).isoformat())

    df_sensors[df_sensors.columns.to_list()[1:]] = df_sensors[df_sensors.columns.to_list()[1:]].astype('Int64')
    
    return df_sensors

In [12]:
# Data Frame Sensors
df_sensors = get_df_sensors(df_tangaras, start_datetime, end_datetime)
df_sensors.head()

,DATETIME,TANGARA_2BBA,TANGARA_14D6,TANGARA_1CE2,TANGARA_1FCA,TANGARA_2492,TANGARA_2FF6,TANGARA_48C6,TANGARA_4D7A,TANGARA_532E,TANGARA_EA06,TANGARA_F1AE,TANGARA_FAC6,TANGARA_06BE
0,2022-09-04T00:00:00-05:00,5,<NA>,4,7,<NA>,<NA>,<NA>,0,0,0,4,1,<NA>
1,2022-09-04T00:00:30-05:00,6,5,4,7,7,7,0,0,0,0,4,1,4
2,2022-09-04T00:01:00-05:00,5,7,4,7,8,7,0,0,2,0,3,1,6
3,2022-09-04T00:01:30-05:00,6,7,4,7,8,8,3,0,0,0,3,1,6
4,2022-09-04T00:02:00-05:00,6,8,4,7,9,8,5,0,2,0,4,1,6


In [13]:
# Check Data Types
df_sensors.dtypes

DATETIME        object
TANGARA_2BBA     Int64
TANGARA_14D6     Int64
TANGARA_1CE2     Int64
TANGARA_1FCA     Int64
TANGARA_2492     Int64
TANGARA_2FF6     Int64
TANGARA_48C6     Int64
TANGARA_4D7A     Int64
TANGARA_532E     Int64
TANGARA_EA06     Int64
TANGARA_F1AE     Int64
TANGARA_FAC6     Int64
TANGARA_06BE     Int64
dtype: object

In [14]:
# Save df_sensors into Catalog
catalog.save('raw_data_sensors_api', df_sensors)

2022-09-05 13:05:36,106 - kedro.io.data_catalog - INFO - Saving data to `raw_data_sensors_api` (CSVDataSet)...
